In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Liver_cirrhosis/GSE139602'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular characterization of chronic liver disease dynamics: from liver fibrosis to acute-on-chronic liver failure"
!Series_summary	"BACKGROUND: The molecular mechanisms driving the progression from early chronic liver disease (eCLD) to cirrhosis and, finally, acute-on-chronic liver failure (ACLF) are largely unknown. Thus, the aim of this work is to develop a network-based approach to investigate molecular pathways driving progression from eCLD to ACLF. We created 9 liver-specific biological networks capturing key pathophysiological processes potentially related to CLD. We used these networks as framework to perform gene set enrichment analyses(GSEA) and create dynamic profiles of disease progression.  RESULTS: Principal component analyses revealed that samples clustered according to the disease stage. GSEA analyses of the defined processes showed an up-regulation of inflammation, fibrosis and apoptosis networks throughout disease progression. I

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

series_summary = '''BACKGROUND: The molecular mechanisms driving the progression from early chronic liver disease (eCLD) to cirrhosis and, finally, acute-on-chronic liver failure (ACLF) are largely unknown. Thus, the aim of this work is to develop a network-based approach to investigate molecular pathways driving progression from eCLD to ACLF. We created 9 liver-specific biological networks capturing key pathophysiological processes potentially related to CLD. We used these networks as framework to perform gene set enrichment analyses(GSEA) and create dynamic profiles of disease progression.  RESULTS: Principal component analyses revealed that samples clustered according to the disease stage. GSEA analyses of the defined processes showed an up-regulation of inflammation, fibrosis and apoptosis networks throughout disease progression. Interestingly, we did not find significant gene expression differences between CC and DC, while ACLF showed acute expression changes in all the defined liver-related networks. The analyses of disease progression patterns identified ascending and descending expression profiles associated to ACLF onset. Functional analyses showed that ascending profiles were associated to inflammation, fibrosis, apoptosis, senescence and carcinogenesis networks, while descending profiles were mainly related to oxidative stress and genetic factors. We confirmed by qPCR the up-regulation of 4 genes of the ascending profile CXCL-6, KRT-18, SPINK-1, and ITGA2, and validated our findings on an independent patient’s cohort.  CONCLUSION: ACLF is characterized by a specific hepatic gene expression pattern related to inflammation, fibrosis, apoptosis, senescence and carcinogenesis processes. Moreover, the observed profile is significantly different from that of compensated and decompensated cirrhosis, supporting thus the hypothesis that ACLF should be considered a distinct entity.'''

if 'gene expression pattern' in series_summary:
    is_gene_available = True

sample_characteristics = {
    0: ['disease state: Healthy', 'disease state: eCLD', 'disease state: Compensated Cirrhosis', 'disease state: Decompensated Cirrhosis', 'disease state: Acute-on-chronic liver failure']
}

# Determine rows for 'Liver_cirrhosis', 'age', and 'gender'
if len(set(sample_characteristics[0])) > 1:
    trait_row = 0

# Age and gender data not available from the given output, so they remain None.
age_row = None
gender_row = None

# Conversion functions
def convert_trait(value):
    mapping = {
        'Healthy': 0,
        'eCLD': 0.5,  # Differentiate early CLD as a mid-stage
        'Compensated Cirrhosis': 1,
        'Decompensated Cirrhosis': 1,
        'Acute-on-chronic liver failure': 1
    }
    try:
        return mapping[value.split(': ')[1]]
    except KeyError:
        return None

def convert_age(value):
    try:
        return float(value.split(': ')[1])
    except ValueError:
        return None

def convert_gender(value):
    gender_mapping = {'male': 1, 'female': 0}
    try:
        return gender_mapping[value.split(': ')[1].lower()]
    except KeyError:
        return None

# Save cohort information
save_cohort_info('GSE139602', './preprocessed/Liver_cirrhosis/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical feature extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Liver_cirrhosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Liver_cirrhosis/trait_data/GSE139602.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM4144550': [0], 'GSM4144551': [0], 'GSM4144552': [0], 'GSM4144553': [0], 'GSM4144554': [0], 'GSM4144555': [0], 'GSM4144556': [0.5], 'GSM4144557': [0.5], 'GSM4144558': [0.5], 'GSM4144559': [0.5], 'GSM4144560': [0.5], 'GSM4144561': [1], 'GSM4144562': [1], 'GSM4144563': [1], 'GSM4144564': [1], 'GSM4144565': [1], 'GSM4144566': [1], 'GSM4144567': [1], 'GSM4144568': [1], 'GSM4144569': [None], 'GSM4144570': [None], 'GSM4144571': [None], 'GSM4144572': [None], 'GSM4144573': [None], 'GSM4144574': [None], 'GSM4144575': [None], 'GSM4144576': [None], 'GSM4144577': [None], 'GSM4144578': [None], 'GSM4144579': [None], 'GSM4144580': [None], 'GSM4144581': [1], 'GSM4144582': [1], 'GSM4144583': [1], 'GSM4144584': [1], 'GSM4144585': [1], 'GSM4144586': [1], 'GSM4144587': [1], 'GSM4144588': [1]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at',
       '11715104_s_at', '11715105_at', '11715106_x_at', '11715107_s_at',
       '11715108_x_at', '11715109_at', '11715110_at', '11715111_s_at',
       '11715112_at', '11715113_x_at', '11715114_x_at', '11715115_s_at',
       '11715116_s_at', '11715117_x_at', '11715118_s_at', '11715119_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['11715100_at', '11715101_s_at', '11715102_x_at', '11715103_x_at', '11715104_s_at'], 'GeneChip Array': ['Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array', 'Human Genome HG-U219 Array'], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10', '20-Aug-10'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database', 'Affymetrix Proprietary Database'], 'Transcript ID(Array Design)': ['g21264570', 'g21264570', 'g21264570', 'g22748780', 'g30039713'], 'Target Description': ['g21264570 /TID=g21264570 /CNT=1 /FEA=FLmRNA /TIER=FL /STK=0 /DEF=g21264

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Decide which key stores identifiers and which key stores gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Gene Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Liver_cirrhosis/gene_data/GSE139602.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Liver_cirrhosis')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE139602', './preprocessed/Liver_cirrhosis/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Liver_cirrhosis/GSE139602.csv'
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'Liver_cirrhosis':
  25%: 0.5
  50% (Median): 1.0
  75%: 1.0
Min: 0.0
Max: 1.0
The distribution of the feature 'Liver_cirrhosis' in this dataset is fine.

